In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from typing import Tuple
from torch.utils.data import Dataset, random_split
import copy
import copy
import torch
from torch.optim import lr_scheduler

In [2]:
class TaskDataset(Dataset):
    def __init__(self, transform=None):
        self.ids = []
        self.imgs = []
        self.labels = []
        self.transform = transform

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        label = self.labels[index]
        return id_, img, label

    def __len__(self):
        return len(self.ids)


class MembershipDataset(TaskDataset):
    def __init__(self, transform=None):
        super().__init__(transform)
        self.membership = []

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int, int]:
        id_, img, label = super().__getitem__(index)
        return id_, img, label, self.membership[index]

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
])
public_dataset: MembershipDataset(transform=transform) = torch.load("./pub.pt")
train_size = int(0.8 * len(public_dataset))
test_size = len(public_dataset) - train_size

train_dataset, test_dataset = random_split(public_dataset, [train_size, test_size])

In [4]:
def train(model, optimizer, criterion, num_epochs=50, patience=5):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = float('inf')
    early_stop_counter = 0
    
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
        for _, inputs, labels, _ in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for _, inputs, labels, _ in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
        
        val_loss /= len(val_loader.dataset)
        print(f'Validation Loss: {val_loss:.4f}')
        
        scheduler.step(val_loss)  # Adjust learning rate based on validation loss

        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print('Early stopping')
            break

    model.load_state_dict(best_model_wts)
    return model


In [8]:
shadow_model_res18 = models.resnet18(pretrained=False)
shadow_model_res18.fc = nn.Linear(512, 44)
criterion_res18 = nn.CrossEntropyLoss()
optimizer_res18 = optim.Adam(shadow_model_res18.parameters(), lr=0.001)

train(shadow_model_res18, optimizer_res18, criterion_res18)

/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/50, Loss: 1.5530
Validation Loss: 2.0592
Epoch 2/50, Loss: 1.2573
Validation Loss: 1.8164
Epoch 3/50, Loss: 1.1409
Validation Loss: 1.7180
Epoch 4/50, Loss: 1.0772
Validation Loss: 2.3311
Epoch 5/50, Loss: 1.0124
Validation Loss: 1.3756
Epoch 6/50, Loss: 0.9711
Validation Loss: 1.5161
Epoch 7/50, Loss: 0.9329
Validation Loss: 1.1818
Epoch 8/50, Loss: 0.8859
Validation Loss: 1.3681
Epoch 9/50, Loss: 0.8509
Validation Loss: 0.9411
Epoch 10/50, Loss: 0.8096
Validation Loss: 1.2662
Epoch 11/50, Loss: 0.7733
Validation Loss: 1.2751
Epoch 12/50, Loss: 0.7326
Validation Loss: 1.0821
Epoch 13/50, Loss: 0.6977
Validation Loss: 1.1847
Epoch 14/50, Loss: 0.5157
Validation Loss: 0.9131
Epoch 15/50, Loss: 0.4533
Validation Loss: 0.9631
Epoch 16/50, Loss: 0.4146
Validation Loss: 1.0105
Epoch 17/50, Loss: 0.3866
Validation Loss: 1.0284
Epoch 18/50, Loss: 0.3552
Validation Loss: 1.1040
Epoch 19/50, Loss: 0.3159
Validation Loss: 1.0931
Early stopping


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
torch.save(shadow_model_res18, "shadow_model_res18.pt")

In [10]:
shadow_model_vgg = models.vgg16(pretrained=False)
shadow_model_vgg.classifier[6] = nn.Linear(4096, 44)

criterion_vgg = nn.CrossEntropyLoss()
optimizer_vgg = optim.Adam(shadow_model_vgg.parameters(), lr=0.001)
train(shadow_model_vgg, optimizer_vgg, criterion_vgg)

Epoch 1/50, Loss: 2.9762
Validation Loss: 2.2349
Epoch 2/50, Loss: 1.8814
Validation Loss: 1.6009
Epoch 3/50, Loss: 1.6368
Validation Loss: 1.7253
Epoch 4/50, Loss: 1.5104
Validation Loss: 1.4479
Epoch 5/50, Loss: 1.4542
Validation Loss: 1.3591
Epoch 6/50, Loss: 1.4134
Validation Loss: 1.4860
Epoch 7/50, Loss: 1.3492
Validation Loss: 1.2767
Epoch 8/50, Loss: 1.2953
Validation Loss: 1.2174
Epoch 9/50, Loss: 1.2729
Validation Loss: 1.2107
Epoch 10/50, Loss: 1.2317
Validation Loss: 1.2382
Epoch 11/50, Loss: 1.2218
Validation Loss: 1.1668
Epoch 12/50, Loss: 1.1792
Validation Loss: 1.2319
Epoch 13/50, Loss: 1.1509
Validation Loss: 1.1698
Epoch 14/50, Loss: 1.1049
Validation Loss: 1.1460
Epoch 15/50, Loss: 1.0678
Validation Loss: 1.1354
Epoch 16/50, Loss: 1.0352
Validation Loss: 1.1872
Epoch 17/50, Loss: 1.0355
Validation Loss: 1.2006
Epoch 18/50, Loss: 0.9938
Validation Loss: 1.1536
Epoch 19/50, Loss: 0.9712
Validation Loss: 1.1170
Epoch 20/50, Loss: 0.9496
Validation Loss: 1.1252
Epoch 21/

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
torch.save(shadow_model_vgg, "shadow_model_vgg.pt")

In [12]:
shadow_model_dense = models.densenet121(pretrained=False)
shadow_model_dense.classifier = nn.Linear(1024, 44)

criterion_dense = nn.CrossEntropyLoss()
optimizer_dense = optim.Adam(shadow_model_dense.parameters(), lr=0.001)

train(shadow_model_dense, optimizer_dense, criterion_dense)

Epoch 1/50, Loss: 1.4950
Validation Loss: 1.5266
Epoch 2/50, Loss: 1.1767
Validation Loss: 1.4094
Epoch 3/50, Loss: 1.0624
Validation Loss: 1.1106
Epoch 4/50, Loss: 0.9901
Validation Loss: 1.1987
Epoch 5/50, Loss: 0.9365
Validation Loss: 1.0234
Epoch 6/50, Loss: 0.8827
Validation Loss: 1.2721
Epoch 7/50, Loss: 0.8435
Validation Loss: 1.1304
Epoch 8/50, Loss: 0.8013
Validation Loss: 1.2452
Epoch 9/50, Loss: 0.7605
Validation Loss: 1.1126
Epoch 10/50, Loss: 0.5818
Validation Loss: 0.7767
Epoch 11/50, Loss: 0.5169
Validation Loss: 0.7912
Epoch 12/50, Loss: 0.4847
Validation Loss: 0.8183
Epoch 13/50, Loss: 0.4503
Validation Loss: 0.8357
Epoch 14/50, Loss: 0.4139
Validation Loss: 0.8736
Epoch 15/50, Loss: 0.3630
Validation Loss: 0.8551
Early stopping


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [13]:
torch.save(shadow_model_dense, "shadow_model_dense.pt")

In [14]:
shadow_model_resnext = models.resnext50_32x4d(pretrained=False)
shadow_model_resnext.fc = nn.Linear(2048, 44)

criterion_resnext = nn.CrossEntropyLoss()
optimizer_resnext = optim.Adam(shadow_model_resnext.parameters(), lr=0.001)

train(shadow_model_resnext, optimizer_resnext, criterion_resnext)

Epoch 1/50, Loss: 2.0037
Validation Loss: 10.2342
Epoch 2/50, Loss: 1.6413
Validation Loss: 5.6467
Epoch 3/50, Loss: 1.7932
Validation Loss: 11.4393
Epoch 4/50, Loss: 1.4697
Validation Loss: 1.5470
Epoch 5/50, Loss: 1.2543
Validation Loss: 1.7732
Epoch 6/50, Loss: 1.1729
Validation Loss: 1.2368
Epoch 7/50, Loss: 1.1357
Validation Loss: 1.8642
Epoch 8/50, Loss: 1.0491
Validation Loss: 1.5276
Epoch 9/50, Loss: 0.9851
Validation Loss: 1.6394
Epoch 10/50, Loss: 0.9708
Validation Loss: 1.1344
Epoch 11/50, Loss: 0.9062
Validation Loss: 1.1138
Epoch 12/50, Loss: 0.8694
Validation Loss: 1.3819
Epoch 13/50, Loss: 0.8135
Validation Loss: 1.3936
Epoch 14/50, Loss: 0.7920
Validation Loss: 1.5964
Epoch 15/50, Loss: 0.7698
Validation Loss: 1.0132
Epoch 16/50, Loss: 0.6947
Validation Loss: 1.0507
Epoch 17/50, Loss: 0.6545
Validation Loss: 1.0850
Epoch 18/50, Loss: 0.6264
Validation Loss: 1.2243
Epoch 19/50, Loss: 0.5643
Validation Loss: 1.1586
Epoch 20/50, Loss: 0.3729
Validation Loss: 1.1275
Early s

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [15]:
torch.save(shadow_model_resnext, "shadow_model_resnext.pt")

In [16]:
shadow_model_effnet = models.efficientnet_b0(pretrained=False)
shadow_model_effnet.classifier[1] = nn.Linear(1280, 44)

criterion_effnet = nn.CrossEntropyLoss()
optimizer_effnet = optim.Adam(shadow_model_effnet.parameters(), lr=0.001)

train(shadow_model_effnet, optimizer_effnet, criterion_effnet)

Epoch 1/50, Loss: 1.9655
Validation Loss: 1.4605
Epoch 2/50, Loss: 1.4400
Validation Loss: 1.3719
Epoch 3/50, Loss: 1.2758
Validation Loss: 1.2112
Epoch 4/50, Loss: 1.1994
Validation Loss: 1.2427
Epoch 5/50, Loss: 1.1267
Validation Loss: 1.1291
Epoch 6/50, Loss: 1.0802
Validation Loss: 1.1523
Epoch 7/50, Loss: 1.0155
Validation Loss: 1.1238
Epoch 8/50, Loss: 0.9702
Validation Loss: 1.2550
Epoch 9/50, Loss: 0.9312
Validation Loss: 1.0495
Epoch 10/50, Loss: 0.8734
Validation Loss: 1.0613
Epoch 11/50, Loss: 0.8195
Validation Loss: 0.9529
Epoch 12/50, Loss: 0.8054
Validation Loss: 1.0489
Epoch 13/50, Loss: 0.7550
Validation Loss: 1.0485
Epoch 14/50, Loss: 0.7300
Validation Loss: 1.0301
Epoch 15/50, Loss: 0.6825
Validation Loss: 1.0132
Epoch 16/50, Loss: 0.5241
Validation Loss: 1.0091
Early stopping


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [17]:
torch.save(shadow_model_effnet, "shadow_model_effnet.pt")

In [18]:
shadow_model_mobilenet = models.mobilenet_v2(pretrained=False)
shadow_model_mobilenet.classifier[1] = nn.Linear(1280, 44)

criterion_mobilenet = nn.CrossEntropyLoss()
optimizer_mobilenet = optim.Adam(shadow_model_mobilenet.parameters(), lr=0.001)

train(shadow_model_mobilenet, optimizer_mobilenet, criterion_mobilenet)


Epoch 1/50, Loss: 1.9011
Validation Loss: 1.6533
Epoch 2/50, Loss: 1.4820
Validation Loss: 1.4477
Epoch 3/50, Loss: 1.3380
Validation Loss: 1.2304
Epoch 4/50, Loss: 1.2358
Validation Loss: 1.2819
Epoch 5/50, Loss: 1.1977
Validation Loss: 1.2154
Epoch 6/50, Loss: 1.1337
Validation Loss: 1.1620
Epoch 7/50, Loss: 1.1004
Validation Loss: 1.1028
Epoch 8/50, Loss: 1.0594
Validation Loss: 1.0694
Epoch 9/50, Loss: 1.0263
Validation Loss: 1.1257
Epoch 10/50, Loss: 1.0017
Validation Loss: 1.0442
Epoch 11/50, Loss: 0.9542
Validation Loss: 1.1876
Epoch 12/50, Loss: 0.9324
Validation Loss: 1.1347
Epoch 13/50, Loss: 0.9092
Validation Loss: 1.0181
Epoch 14/50, Loss: 0.8862
Validation Loss: 1.1168
Epoch 15/50, Loss: 0.8583
Validation Loss: 1.1478
Epoch 16/50, Loss: 0.8400
Validation Loss: 1.1247
Epoch 17/50, Loss: 0.8077
Validation Loss: 0.9856
Epoch 18/50, Loss: 0.7850
Validation Loss: 1.1112
Epoch 19/50, Loss: 0.7654
Validation Loss: 1.0531
Epoch 20/50, Loss: 0.7300
Validation Loss: 1.1767
Epoch 21/

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [19]:
torch.save(shadow_model_mobilenet, "shadow_model_mobilenet.pt")

In [5]:
shadow_model_res18 = models.resnet18(pretrained=False)
shadow_model_res18.fc = nn.Linear(512, 44)
criterion_res18 = nn.CrossEntropyLoss()
optimizer_res18 = optim.SGD(shadow_model_res18.parameters(), lr=0.01, momentum=0.9)

train(shadow_model_res18, optimizer_res18, criterion_res18)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/50, Loss: 1.8837
Validation Loss: 1.5029
Epoch 2/50, Loss: 1.3018
Validation Loss: 1.4210
Epoch 3/50, Loss: 1.1478
Validation Loss: 1.3163
Epoch 4/50, Loss: 1.0547
Validation Loss: 1.2778
Epoch 5/50, Loss: 0.9863
Validation Loss: 1.2658
Epoch 6/50, Loss: 0.9279
Validation Loss: 1.1917
Epoch 7/50, Loss: 0.8687
Validation Loss: 1.2259
Epoch 8/50, Loss: 0.8044
Validation Loss: 1.2117
Epoch 9/50, Loss: 0.7524
Validation Loss: 1.2482
Epoch 10/50, Loss: 0.6996
Validation Loss: 1.2126
Epoch 11/50, Loss: 0.5382
Validation Loss: 1.0509
Epoch 12/50, Loss: 0.4844
Validation Loss: 1.0623
Epoch 13/50, Loss: 0.4610
Validation Loss: 1.0649
Epoch 14/50, Loss: 0.4431
Validation Loss: 1.0765
Epoch 15/50, Loss: 0.4249
Validation Loss: 1.0874
Epoch 16/50, Loss: 0.4010
Validation Loss: 1.0846
Early stopping


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
torch.save(shadow_model_res18, "shadow_model_res18_sgd.pt")

In [7]:
shadow_model_dense = models.densenet121(pretrained=False)
shadow_model_dense.classifier = nn.Linear(1024, 44)
criterion_dense = nn.CrossEntropyLoss()
optimizer_dense = optim.AdamW(shadow_model_dense.parameters(), lr=0.001)

train(shadow_model_dense, optimizer_dense, criterion_dense)


Epoch 1/50, Loss: 1.4571
Validation Loss: 1.6035
Epoch 2/50, Loss: 1.1602
Validation Loss: 1.3666
Epoch 3/50, Loss: 1.0551
Validation Loss: 1.1778
Epoch 4/50, Loss: 0.9815
Validation Loss: 1.1235
Epoch 5/50, Loss: 0.9206
Validation Loss: 1.1505
Epoch 6/50, Loss: 0.8789
Validation Loss: 1.3355
Epoch 7/50, Loss: 0.8442
Validation Loss: 1.0945
Epoch 8/50, Loss: 0.7990
Validation Loss: 1.0020
Epoch 9/50, Loss: 0.7581
Validation Loss: 1.1114
Epoch 10/50, Loss: 0.7206
Validation Loss: 1.2644
Epoch 11/50, Loss: 0.6780
Validation Loss: 0.9904
Epoch 12/50, Loss: 0.6390
Validation Loss: 0.9271
Epoch 13/50, Loss: 0.5916
Validation Loss: 1.2158
Epoch 14/50, Loss: 0.5625
Validation Loss: 1.2455
Epoch 15/50, Loss: 0.5099
Validation Loss: 1.1311
Epoch 16/50, Loss: 0.4757
Validation Loss: 1.2802
Epoch 17/50, Loss: 0.2929
Validation Loss: 0.9768
Early stopping


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [8]:
torch.save(shadow_model_dense, "shadow_model_dense_adamw.pt")

In [13]:
shadow_model_effnet = models.efficientnet_b0(pretrained=False)
shadow_model_effnet.classifier[1] = nn.Linear(1280, 44)
criterion_effnet = nn.CrossEntropyLoss()
optimizer_effnet = optim.RMSprop(shadow_model_effnet.parameters(), lr=0.001, alpha=0.9)

train(shadow_model_effnet, optimizer_effnet, criterion_effnet)


In [ ]:
torch.save(shadow_model_effnet, "shadow_model_effnet_rms.pt")

In [6]:
shadow_model_shufflenet = models.shufflenet_v2_x1_0(pretrained=False)
shadow_model_shufflenet.fc = nn.Linear(1024, 44)
criterion_shufflenet = nn.CrossEntropyLoss()
optimizer_shufflenet = optim.Adam(shadow_model_shufflenet.parameters(), lr=0.001)

train(shadow_model_shufflenet, optimizer_shufflenet, criterion_shufflenet)


Epoch 1/50, Loss: 1.7919
Validation Loss: 1.5156
Epoch 2/50, Loss: 1.3775
Validation Loss: 1.3706
Epoch 3/50, Loss: 1.2428
Validation Loss: 1.2802
Epoch 4/50, Loss: 1.1568
Validation Loss: 1.3230
Epoch 5/50, Loss: 1.0938
Validation Loss: 1.2330
Epoch 6/50, Loss: 1.0429
Validation Loss: 1.2584
Epoch 7/50, Loss: 0.9952
Validation Loss: 1.3649
Epoch 8/50, Loss: 0.9586
Validation Loss: 1.1341
Epoch 9/50, Loss: 0.9246
Validation Loss: 1.1434
Epoch 10/50, Loss: 0.8753
Validation Loss: 1.1233
Epoch 11/50, Loss: 0.8483
Validation Loss: 1.1346
Epoch 12/50, Loss: 0.8209
Validation Loss: 1.1970
Epoch 13/50, Loss: 0.7878
Validation Loss: 1.1140
Epoch 14/50, Loss: 0.7510
Validation Loss: 1.2397
Epoch 15/50, Loss: 0.7267
Validation Loss: 1.1775
Epoch 16/50, Loss: 0.6886
Validation Loss: 1.1143
Epoch 17/50, Loss: 0.6660
Validation Loss: 1.2155
Epoch 18/50, Loss: 0.4926
Validation Loss: 1.1430
Early stopping


ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_

In [7]:
torch.save(shadow_model_shufflenet, "shadow_model_shufflenet.pt")

In [8]:
shadow_model_res34 = models.resnet34(pretrained=False)
shadow_model_res34.fc = nn.Linear(512, 44)
criterion_res34 = nn.CrossEntropyLoss()
optimizer_res34 = optim.Adam(shadow_model_res34.parameters(), lr=0.001)

train(shadow_model_res34, optimizer_res34, criterion_res34)


Epoch 1/50, Loss: 1.6475
Validation Loss: 1.8088
Epoch 2/50, Loss: 1.3398
Validation Loss: 1.7387
Epoch 3/50, Loss: 1.2378
Validation Loss: 2.1826
Epoch 4/50, Loss: 1.1663
Validation Loss: 1.4122
Epoch 5/50, Loss: 1.1027
Validation Loss: 1.1772
Epoch 6/50, Loss: 1.0535
Validation Loss: 1.6924
Epoch 7/50, Loss: 0.9975
Validation Loss: 1.4690
Epoch 8/50, Loss: 0.9552
Validation Loss: 1.2207
Epoch 9/50, Loss: 0.9069
Validation Loss: 1.5950
Epoch 10/50, Loss: 0.7433
Validation Loss: 0.9009
Epoch 11/50, Loss: 0.6840
Validation Loss: 0.9000
Epoch 12/50, Loss: 0.6512
Validation Loss: 0.8970
Epoch 13/50, Loss: 0.6330
Validation Loss: 0.8963
Epoch 14/50, Loss: 0.5979
Validation Loss: 0.9211
Epoch 15/50, Loss: 0.5733
Validation Loss: 1.0034
Epoch 16/50, Loss: 0.5355
Validation Loss: 0.9639
Epoch 17/50, Loss: 0.5122
Validation Loss: 1.0206
Epoch 18/50, Loss: 0.4656
Validation Loss: 1.0037
Early stopping


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
torch.save(shadow_model_res34, "shadow_model_res34.pt")

In [11]:
shadow_model_squeezenet = models.squeezenet1_0(pretrained=False)
shadow_model_squeezenet.classifier[1] = nn.Conv2d(512, 44, kernel_size=(1, 1), stride=(1, 1))
criterion_squeezenet = nn.CrossEntropyLoss()
optimizer_squeezenet = optim.Adam(shadow_model_squeezenet.parameters(), lr=0.001)

train(shadow_model_squeezenet, optimizer_squeezenet, criterion_squeezenet)


Epoch 1/50, Loss: 2.4755
Validation Loss: 2.1678
Epoch 2/50, Loss: 2.1263
Validation Loss: 2.0144
Epoch 3/50, Loss: 2.0130
Validation Loss: 1.9308
Epoch 4/50, Loss: 1.9584
Validation Loss: 1.9441
Epoch 5/50, Loss: 1.9028
Validation Loss: 1.8336
Epoch 6/50, Loss: 1.8661
Validation Loss: 1.8897
Epoch 7/50, Loss: 1.8430
Validation Loss: 1.8212
Epoch 8/50, Loss: 1.8081
Validation Loss: 1.8025
Epoch 9/50, Loss: 1.7916
Validation Loss: 1.8333
Epoch 10/50, Loss: 1.7814
Validation Loss: 1.7773
Epoch 11/50, Loss: 1.7556
Validation Loss: 1.8853
Epoch 12/50, Loss: 1.7397
Validation Loss: 1.7616
Epoch 13/50, Loss: 1.7122
Validation Loss: 1.8206
Epoch 14/50, Loss: 1.6943
Validation Loss: 1.7447
Epoch 15/50, Loss: 1.6678
Validation Loss: 1.8065
Epoch 16/50, Loss: 1.6653
Validation Loss: 1.7579
Epoch 17/50, Loss: 1.6351
Validation Loss: 1.7478
Epoch 18/50, Loss: 1.6413
Validation Loss: 1.7623
Epoch 19/50, Loss: 1.4934
Validation Loss: 1.7082
Epoch 20/50, Loss: 1.4399
Validation Loss: 1.7233
Epoch 21/

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [12]:
torch.save(shadow_model_squeezenet, "shadow_model_squeezenet.pt")